In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import wandb

# Replace 'your-api-key' with your actual API key
wandb.login(key='d1edd05dd769529255223579346f240eda999b02')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shshohan0000 (symom). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import os
import shutil
import pandas as pd
import numpy as np
import torch

from torch.utils.data import Dataset
from sklearn.metrics import mean_absolute_error
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

################################################################################
# 1. Choose a (Multi)lingual model
################################################################################
MODEL_CHECKPOINT = "FacebookAI/xlm-roberta-base"

################################################################################
# 2. List of language codes
################################################################################
LANG_CODES = [
     "amh", "arq", "chn", "deu", "esp", "hau",
     "ptbr", "ron",
    "rus",  "ukr"
]

################################################################################
# 3. Base paths (adjust to Track B data)
################################################################################
BASE_PATH = "/kaggle/input/semeval-t11/public_data_test/track_b"  # Changed to track_b
TRAIN_FOLDER = os.path.join(BASE_PATH, "train")
DEV_FOLDER = os.path.join(BASE_PATH, "dev")
TEST_FOLDER = os.path.join(BASE_PATH, "test")

################################################################################
# 4. Custom Dataset class for regression
################################################################################
class EmotionsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {k: v.squeeze() for k, v in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.float)  # Labels as floats
        return item

################################################################################
# 5. Initialize tokenizer once
################################################################################
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

################################################################################
# 6. Loop over each language
################################################################################
for lang in LANG_CODES:
    print(f"=== PROCESSING LANGUAGE: {lang} ===")

    # File paths
    train_path = os.path.join(TRAIN_FOLDER, f"{lang}.csv")
    dev_path = os.path.join(DEV_FOLDER, f"{lang}.csv")
    test_path = os.path.join(TEST_FOLDER, f"{lang}.csv")

    # Read data
    df_train = pd.read_csv(train_path)
    df_dev = pd.read_csv(dev_path)
    df_test = pd.read_csv(test_path)

    # Emotion columns (Track B includes all six emotions)
    emotion_columns = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
    col_map = {c.lower(): c for c in df_train.columns}
    actual_emotion_cols = [col_map[emo.lower()] for emo in emotion_columns]

    # Extract texts and labels
    train_texts = df_train["text"].tolist()
    train_labels = df_train[actual_emotion_cols].values.astype(float)

    dev_texts = df_dev["text"].tolist()
    dev_labels = df_dev[actual_emotion_cols].values.astype(float)

    # Test dummy labels (unused but needed for dataset)
    test_texts = df_test["text"].tolist()
    test_dummy_labels = np.zeros((len(df_test), len(actual_emotion_cols)))

    # Create datasets
    train_dataset = EmotionsDataset(train_texts, train_labels, tokenizer)
    dev_dataset = EmotionsDataset(dev_texts, dev_labels, tokenizer)
    test_dataset = EmotionsDataset(test_texts, test_dummy_labels, tokenizer)

    # Define metrics function inside loop to capture emotion columns
    def compute_metrics(eval_pred, emotion_cols=actual_emotion_cols):
        predictions, labels = eval_pred
        rounded_preds = np.round(predictions)
        rounded_preds = np.clip(rounded_preds, 0, 3)
        mae = mean_absolute_error(labels, rounded_preds)
        exact_match = np.all(rounded_preds == labels, axis=1).mean()
        emotion_maes = {
            f"{col}_mae": mean_absolute_error(labels[:, i], rounded_preds[:, i])
            for i, col in enumerate(emotion_cols)
        }
        return {"mae": mae, "exact_match": exact_match, **emotion_maes}

    # Load model for regression
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT,
        num_labels=len(actual_emotion_cols),
        problem_type="regression"
    )

    # Training arguments
    output_dir = f"outputs_{lang}_trackb"
    logs_dir = f"logs_{lang}_trackb"

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        logging_dir=logs_dir,
        logging_steps=50,
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    print(f"\n--- Training model for: {lang} ---")
    trainer.train()

    print(f"--- Evaluation on dev set ({lang}) ---")
    trainer.evaluate()

    # Predict on test set
    print(f"--- Generating predictions for: {lang} ---")
    test_results = trainer.predict(test_dataset)
    predictions = test_results.predictions
    rounded_preds = np.round(predictions).astype(int)
    rounded_preds = np.clip(rounded_preds, 0, 3)

    # Update test DataFrame
    for i, col in enumerate(actual_emotion_cols):
        df_test[col] = rounded_preds[:, i]

    # Save predictions
    out_filename = f"pred_{lang}.csv"
    df_test= df_test.drop("text", axis=1)
    df_test.to_csv(out_filename, index=False)
    print(f"Saved predictions to {out_filename}")

    # Cleanup
    shutil.rmtree(output_dir, ignore_errors=True)
    shutil.rmtree(logs_dir, ignore_errors=True)
    print(f"Deleted {output_dir} and {logs_dir}.\n")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

=== PROCESSING LANGUAGE: amh ===


...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-3-b1dcab687d16>:145: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



--- Training model for: ukr ---


Epoch,Training Loss,Validation Loss,Mae,Exact Match,Anger Mae,Disgust Mae,Fear Mae,Joy Mae,Sadness Mae,Surprise Mae
1,0.208000,0.168232,0.129853,0.461847,0.080321,0.048193,0.092369,0.248996,0.208835,0.100402
2,0.158700,0.155106,0.127175,0.510040,0.080321,0.048193,0.096386,0.248996,0.188755,0.100402
3,0.149600,0.150949,0.123159,0.534137,0.080321,0.048193,0.064257,0.248996,0.188755,0.108434
4,0.118500,0.156129,0.131861,0.502008,0.080321,0.048193,0.116466,0.228916,0.216867,0.100402
5,0.116100,0.156413,0.125167,0.530120,0.080321,0.048193,0.092369,0.257028,0.176707,0.096386


--- Evaluation on dev set (ukr) ---


--- Generating predictions for: ukr ---
Saved predictions to pred_ukr.csv
Deleted outputs_ukr_trackb and logs_ukr_trackb.

